In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_04 = pd.read_csv(folderpath + 'data_op_04.csv')
data_op_04.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
4,1,5,42.0041,0.8405,40.0,1354.48,1124.32,3.91,2211.80,130.44,2387.89,0.02,4,1
6,1,7,41.9998,0.8400,40.0,1354.43,1131.44,3.91,2211.82,130.16,2387.88,0.02,4,1
9,1,10,42.0066,0.8400,40.0,1353.19,1125.09,3.91,2211.84,130.32,2387.91,0.02,4,1
12,1,13,42.0029,0.8409,40.0,1350.90,1123.53,3.91,2211.78,130.60,2387.88,0.02,4,1
20,1,21,42.0030,0.8404,40.0,1349.16,1117.33,3.91,2211.82,130.48,2387.90,0.02,4,1


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_04 = tf_remove_sensor_15.fit_transform(data_op_04)
data_op_04.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
4,1,5,42.0041,0.8405,40.0,1354.48,1124.32,3.91,2211.80,130.44,2387.89,4,1
6,1,7,41.9998,0.8400,40.0,1354.43,1131.44,3.91,2211.82,130.16,2387.88,4,1
9,1,10,42.0066,0.8400,40.0,1353.19,1125.09,3.91,2211.84,130.32,2387.91,4,1
12,1,13,42.0029,0.8409,40.0,1350.90,1123.53,3.91,2211.78,130.60,2387.88,4,1
20,1,21,42.0030,0.8404,40.0,1349.16,1117.33,3.91,2211.82,130.48,2387.90,4,1


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_04)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
4,1354.48,1124.32,3.91,2211.80,130.44,2387.89,1
6,1354.43,1131.44,3.91,2211.82,130.16,2387.88,1
9,1353.19,1125.09,3.91,2211.84,130.32,2387.91,1
12,1350.90,1123.53,3.91,2211.78,130.60,2387.88,1
20,1349.16,1117.33,3.91,2211.82,130.48,2387.90,1


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[-0.03471011, -0.51398323,  1.        , -0.50007728, -0.27185   ,
        -0.42928696],
       [-0.04353727,  0.4286228 ,  1.        , -0.43380467, -1.08895102,
        -0.45999999],
       [-0.26245095, -0.41204409,  1.        , -0.36753205, -0.62203615,
        -0.36786091],
       ...,
       [ 1.87019199,  2.54551472,  1.        , -2.05748362, -0.94304012,
        -1.96493836],
       [ 1.91962411,  1.85444681,  1.        , -2.19002884, -0.85549359,
        -2.30278167],
       [ 0.80033965,  2.9056114 ,  1.        , -2.35571037, -1.96441641,
        -2.14921653]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 1, ..., 4, 4, 4])

# Classification steps

In [7]:
from phm08ds.models import experiment

## Load best mlp from random search

In [8]:
from sklearn.externals import joblib

random_search_mlp = joblib.load('Results/old_models/clf_mlp.pkl')
# random_search_svm = joblib.load('Results/old_models/clf_svm.pkl')

In [9]:
clf_mlp = random_search_mlp['MLP'].best_estimator_

## Put all clf in a dictionary:

In [10]:
classifiers = {'MLP': clf_mlp}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

In [11]:
kfolds = 10
clf_outputs = experiment.run_classifiers(data_with_features_std, labels, classifiers, kfolds)

## Performance assessment

Saving variables in a dictionary:

In [12]:
results = {}
results['train'] = experiment.results_clf(4, clf_outputs['train']['true'], clf_outputs['train']['pred'])
results['test'] = experiment.results_clf(4, clf_outputs['test']['true'], clf_outputs['test']['pred'])

## Savel results, models and pipeline to a .pkl file 

In [13]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [14]:
experiment.save_models(classifiers)
experiment.save_pipeline(data_preprocessing)

## Save results to CSVs and figures

In [15]:
experiment.export_results(results['test'], 'test')
experiment.export_results(results['train'], 'train')

/mnt/Work/Mestrado/Mestrado/MP-Safety_ITA/Analyses_and_Experiments/PHM08_data_science/notebooks/E03_PHM08-train-best_MLP_and_SVM/model_selection-OP_04-navarmn
MLP
/mnt/Work/Mestrado/Mestrado/MP-Safety_ITA/Analyses_and_Experiments/PHM08_data_science/notebooks/E03_PHM08-train-best_MLP_and_SVM/model_selection-OP_04-navarmn
MLP


## Savel results, models and pipeline to a .pkl file 

In [16]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [17]:
experiment.save_models(clf_mlp, name='clf_mlp')
experiment.save_pipeline(data_preprocessing)